In [14]:
import geopandas as gpd
import pandas as pd
from pathlib import Path
import sys
from loguru import logger

logger.remove()
logger.add(sys.stderr, level="INFO")
data_path = Path("../data")
# data_path = Path('data')
config = pd.read_excel(data_path / "config.xlsx", sheet_name="layers")
config.tail(3)

,Name,shape_type,color,size,alpha,label,file_type,shp,gdb,layer,display
22,Pipeline ESMT,line,blue,2.0,0.5,NaN,shp,\\ppeng.com\pzdata\clients\Arvin-Edison WSD-12...,NaN,NaN,1.0
23,"SL,PL,R-W",line,#FF00FF,2.0,0.2,NaN,shp,\\ppeng.com\pzdata\clients\Arvin-Edison WSD-12...,NaN,NaN,1.0
24,Proposed Pipeline,line,orange,4.0,1.0,Label,shp,\\ppeng.com\pzdata\clients\Arvin-Edison WSD-12...,NaN,NaN,1.0


In [9]:
# row = config.iloc[-1]
# gpd.read_file(Path(row["shp"]))

In [15]:
def get_layer(row):
    try:
        logger.info(f"Loading layer {row['Name']}")
        if row["display"] == True:
            if row["file_type"] == "shp":
                gdf = gpd.read_file(Path(row["shp"]))
            elif row["file_type"] == "gdb":
                gdb = Path(row["gdb"])
                gdf = gpd.read_file(gdb, layer=row["layer"])
            else:
                return None

            gdf["color"] = row["color"]
            # gdf["label"] = gdf[row["label"]].astype(str)
            # print(gdf.columns)
            if type(row["label"]) == str:
                gdf["label"] = gdf[row["label"]].astype(str).replace("nan", "")
            else:
                gdf["label"] = ""

            gdf["layer"] = row["Name"]
            gdf["size"] = row["size"]
            # print(row['Name'])
            # print(gdf.crs)

            return gdf[
                ["color", "label", "layer", "size", "geometry"]
            ]  # .to_crs(epsg=4326)
        else:
            logger.info(f"Layer {row['Name']} is not displayed")
            # print(row['Name'])
            return None
    except Exception as e:
        logger.error(f"Layer {row['Name']} failed to load due to {e}")
        return None


# Create gdb
gdfs = {y["Name"]: get_layer(y) for i, y in config.iterrows()}
# gdfs = {y['Name']:y for i,y in config.iterrows()}

# gdfs['Proposed Pipeline']

2024-01-16 16:08:49.718 | INFO     | __main__:get_layer:3 - Loading layer Proposed Pipeline
2024-01-16 16:08:49.719 | INFO     | __main__:get_layer:3 - Loading layer APNs
2024-01-16 16:08:49.720 | INFO     | __main__:get_layer:3 - Loading layer District Boundary
2024-01-16 16:08:49.798 | INFO     | __main__:get_layer:3 - Loading layer AEWSD North Canal
2024-01-16 16:08:49.856 | INFO     | __main__:get_layer:3 - Loading layer FFPPP Discharge Pipeline
2024-01-16 16:08:49.895 | INFO     | __main__:get_layer:3 - Loading layer Frick Unit Service Area
2024-01-16 16:08:49.896 | INFO     | __main__:get_layer:30 - Layer Frick Unit Service Area is not displayed
2024-01-16 16:08:49.897 | INFO     | __main__:get_layer:3 - Loading layer Proposed Turnout
2024-01-16 16:08:49.897 | INFO     | __main__:get_layer:30 - Layer Proposed Turnout is not displayed
2024-01-16 16:08:49.898 | INFO     | __main__:get_layer:3 - Loading layer DiGiorgio Pipeline
2024-01-16 16:08:49.943 | INFO     | __main__:get_layer

In [16]:
epsg = 4326
# epsg = 26745
# epsg = 2229
# epsg = 6424
import arrow

gdf = pd.concat([gdf.to_crs(epsg=epsg) for gdf in gdfs.values() if gdf is not None])

gdf.to_parquet(data_path / f"gdf-{arrow.now().format('YYYY-MM-DD')}.parquet")